Notebook for inputting the final state momenta back into events after reconstruction (from baler), requires a npz of px1,py1,pz1,e1,px2,py2,pz2,e2.

Requries pyhepmc.

In [1]:
import numpy as np
import pyhepmc
from pyhepmc.io import ReaderAsciiHepMC2, WriterAsciiHepMC2
from pyhepmc import GenEvent

def read_npz_and_split_data(input_path):
    data = np.load(input_path)["data"]
    return data[:, :4], data[:, 4:]

def replace_final_state_4_momenta(npz_file, input_file, output_file):
    p1_data, p2_data = read_npz_and_split_data(npz_file)
    count = 0
    with ReaderAsciiHepMC2(input_file) as reader, \
         WriterAsciiHepMC2(output_file) as writer:

        evt = GenEvent()
        while not reader.failed():
            reader.read_event(evt)
            if reader.failed():
                break
            e_pair = []
            for p in evt.particles:
                if p.pid == 23:
                    ev = p.end_vertex
                    for dau in ev.particles_out:
                        if dau.status == 1 and abs(dau.pid) == 11:
                            e_pair.append(dau)
            e_pair[0].momentum = pyhepmc.FourVector(p1_data[count,0], p1_data[count,1], p1_data[count, 2], p1_data[count, 3]) 
            e_pair[1].momentum = pyhepmc.FourVector(p2_data[count,0], p2_data[count,1], p2_data[count, 2], p2_data[count, 3])         

            count += 1                    
            writer.write(evt)

    

In [2]:
replace_final_state_4_momenta("4momenta.npz", "../DATA/HEPMC.43646139._000001.hepmc", "../processedData/reinputed_final_state.hepmc")

FileNotFoundError: [Errno 2] No such file or directory: '4momenta.npz'